# Análisis de Hábitos de Sueño y Construcción de un Modelo Predictivo con Interfaz Web

Este notebook desarrolla un modelo de clasificación para predecir trastornos de sueño utilizando datos de hábitos de vida, estrés y factores relacionados.  
Incluye preprocesamiento, entrenamiento de múltiples modelos, selección del mejor según métricas, análisis de error y una interfaz web para consumo del modelo.


## 1. Instalación de librerías necesarias
Instalamos las librerías requeridas para procesamiento de datos, modelos de clasificación y construcción de una interfaz web.


In [1]:
!pip install pandas numpy scikit-learn gradio


## Cargar los archivos de entrenamiento y prueba


In [2]:
import pandas as pd

train = pd.read_csv("train_sleep_stress.csv")
test = pd.read_csv("test_sleep_stress.csv")

train.head()


,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,BMI Category,Heart Rate,Sleep Disorder,Systolic_BP,Diastolic_BP,Stress_Binary
0,317,Female,53,Engineer,8.5,9,30,Normal,65,NaN,125,80,ESTRES MODERADO
1,283,Female,50,Nurse,6.0,6,90,Overweight,75,Sleep Apnea,140,95,ESTRESADO
2,309,Female,52,Accountant,6.6,7,45,Overweight,72,Insomnia,130,85,ESTRESADO
3,276,Female,49,Nurse,6.2,6,90,Overweight,75,Sleep Apnea,140,95,ESTRESADO
4,243,Male,44,Salesperson,6.4,6,45,Overweight,72,Insomnia,130,85,ESTRESADO


## Preprocesamiento del dataset
Se realiza codificación de variables categóricas, alineación de columnas y separación de variables predictoras y objetivo.


In [3]:
train_encoded = pd.get_dummies(train, drop_first=True)
test_encoded  = pd.get_dummies(test, drop_first=True)

train_encoded, test_encoded = train_encoded.align(test_encoded, join='left', axis=1)
test_encoded = test_encoded.fillna(0)

target_col = train_encoded.columns[-1]

X = train_encoded.drop(target_col, axis=1)
y = train_encoded[target_col]

X_test_final = test_encoded.drop(target_col, axis=1)
y_test_final = test_encoded[target_col]


## Separación del conjunto de entrenamiento y validación


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


## Funciones para métricas adicionales
Se agregan sensibilidad y especificidad para una evaluación más completa.


In [5]:
from sklearn.metrics import classification_report, confusion_matrix

def sensitivity_specificity(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    TN, FP, FN, TP = cm.ravel()
    sensitivity = TP / (TP + FN)
    specificity = TN / (TN + FP)
    return sensitivity, specificity


## Entrenamiento de varios modelos de clasificación
Se comparan cinco modelos: Regresión Logística, Árbol de Decisión, Random Forest, Gradient Boosting y SVM.


In [7]:
results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_val)

    report = classification_report(y_val, preds, output_dict=True)
    sens, spec = sensitivity_specificity(y_val, preds)

    # Extraer métricas de forma segura:
    precision = report.get("weighted avg", {}).get("precision", None)
    recall    = report.get("weighted avg", {}).get("recall", None)
    f1        = report.get("weighted avg", {}).get("f1-score", None)

    # Si weighted avg no existe (solo 1 clase en validación), usar macro avg
    if precision is None:
        precision = report["macro avg"]["precision"]
        recall    = report["macro avg"]["recall"]
        f1        = report["macro avg"]["f1-score"]

    results.append([name, precision, recall, f1, sens, spec])

results_df = pd.DataFrame(
    results,
    columns=["Modelo","Precision","Recall","F1","Sensitivity","Specificity"]
)

results_df


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,Modelo,Precision,Recall,F1,Sensitivity,Specificity
0,LogisticRegression,0.983666,0.982759,0.982882,1.000000,0.975
1,DecisionTree,1.000000,1.000000,1.000000,1.000000,1.000
2,RandomForest,0.983179,0.982759,0.982619,0.944444,1.000
3,GradientBoosting,1.000000,1.000000,1.000000,1.000000,1.000
4,SVM,0.475624,0.689655,0.562984,0.000000,1.000


In [8]:
best_model_name = results_df.sort_values("F1", ascending=False).iloc[0]["Modelo"]
best_model_name


'DecisionTree'

In [9]:
best_model = models[best_model_name]
best_model.fit(X_train, y_train)


DecisionTreeClassifier()

In [10]:
pred_val = best_model.predict(X_val)

errores_mask = (pred_val != y_val)
errores = X_val[errores_mask]

print("Cantidad de errores:", errores_mask.sum())
errores.head()


Cantidad de errores: 0


,Person ID,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Heart Rate,Systolic_BP,Diastolic_BP,Gender_Male,Occupation_Doctor,Occupation_Engineer,Occupation_Lawyer,Occupation_Nurse,Occupation_Salesperson,Occupation_Scientist,Occupation_Software Engineer,Occupation_Teacher,BMI Category_Normal Weight,BMI Category_Overweight,Sleep Disorder_Sleep Apnea


In [11]:
import gradio as gr
import numpy as np

# Crear inputs dinámicos según columnas de X
inputs_list = [gr.Number(label=col) for col in X.columns]

def predictor(*vals):
    arr = np.array(vals).reshape(1, -1)
    pred = best_model.predict(arr)[0]
    return "Trastorno del sueño" if pred == 1 else "Sin trastorno"

demo = gr.Interface(
    fn=predictor,
    inputs=inputs_list,
    outputs="text",
    title="Modelo Predictivo de Trastornos del Sueño",
    description="Ingrese los valores para obtener una predicción basada en el mejor modelo."
)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4d9d9371b1f6664743.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Resumen del Proyecto: Modelo Predictivo de Trastornos del Sueño

Este proyecto desarrolla un modelo de clasificación capaz de predecir si una persona presenta riesgo de trastorno del sueño basándose en variables asociadas al estilo de vida, estrés y hábitos diarios.

## Objetivos del proyecto
- Preparar y limpiar el dataset (preprocesamiento).
- Entrenar varios modelos de clasificación.
- Comparar métricas: Precision, Recall, F1, Sensibilidad y Especificidad.
- Seleccionar automáticamente el mejor modelo.
- Realizar análisis de error para identificar patrones incorrectos.
- Construir una interfaz web con Gradio para probar el modelo interactivamente.

## Modelos probados
- Logistic Regression  
- Decision Tree  
- Random Forest  
- Gradient Boosting  
- SVM  

Se eligió el mejor modelo utilizando como criterio principal el **F1-score**, ya que permite un equilibrio entre precisión y sensibilidad, especialmente útil cuando se trata de predicción de riesgos en salud.

## Resultados
El modelo final se entrena utilizando el conjunto de entrenamiento y la métrica F1 más alta.  
Posteriormente se implementa una interfaz con Gradio donde el usuario puede ingresar los valores de las variables y recibir una predicción del modelo.

## Interfaz Web
La interfaz recibe los datos numéricos del usuario y devuelve:
- “Trastorno del sueño”  
- “Sin trastorno”  

El sistema sirve como una herramienta educativa basada en machine learning y no sustituye evaluaciones médicas reales.

